In [1]:
from rdkit import Chem
from rdkit.Chem import Draw

In [1]:
import numpy as np

In [18]:
ds = np.load('docking_scores.npy', allow_pickle=True)
ds[ds != 0][797:798].min()

-9.8

У train №797 DS -9.8

In [2]:
X = np.load('./data_preprocessed/zinc250k/node_features.npy')
print('Nodes done')
A = np.load('./data_preprocessed/zinc250k/adj_features.npy')
print('Bonds done')

Nodes done
Bonds done


In [16]:
mol_sizes = np.load('./data_preprocessed/zinc1500k/mol_sizes.npy')

In [17]:
max(mol_sizes)

27

In [4]:
import time

In [1]:
import os
current_dir = os.getcwd()  # Текущая рабочая директория
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, 'title.txt')

In [3]:
current_dir, parent_dir, file_path

('q:\\Учеба\\Универ\\6 курс\\Диплом\\NF_mol_gen',
 'q:\\Учеба\\Универ\\6 курс\\Диплом',
 'q:\\Учеба\\Универ\\6 курс\\Диплом\\title.txt')

In [9]:
from preprocess import Zinc_Processor

In [7]:
zp = Zinc_Processor('DMNP_read.txt', './my_scripts')

sl:  ['CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O']


100%|██████████| 1/1 [00:02<00:00,  3.00s/it]

total number of valid molecule in dataset: 1
try to save
./my_scripts
saving node/adj feature...
shape of node feature: (1, 50)
shape of adj features: (1, 3, 50, 50)
shape of mol sizes: (1,)
save config
saving config...
{'atom_list': [6, 7, 8, 9, 15, 16, 17, 35, 53, 0], 'node_dim': 10, 'max_size': 50, 'bond_dim': 4}


In [11]:
np.load('./my_scripts/mol_sizes.npy')

array([19])

In [8]:
from utils import valid_mol

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterCatalogEntry> already registered; second conversion method ignored.


In [7]:
import moses
import pandas as pd

In [2]:
train = moses.get_dataset('train')
test = moses.get_dataset('test')
test_scaffolds = moses.get_dataset('test_scaffolds')

In [2]:
from torch.utils.data import DataLoader
from moses import CharVocab, StringDataset

train = moses.get_dataset('train')
vocab = CharVocab.from_data(train)
train_dataset = StringDataset(vocab, train)
train_dataloader = DataLoader(
    train_dataset, batch_size=512,
    shuffle=True, collate_fn=train_dataset.default_collate
)

In [9]:
pd.Series(train)#.to_csv('zinc1500K.smi', index=False)

0            CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1
1              CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1
2               Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO
3                  Cn1cnc2c1c(=O)n(CC(O)CO)c(=O)n2C
4                    CC1Oc2ccc(Cl)cc2N(CC(O)CO)C1=O
                             ...                   
1584658                        N#Cc1c(Br)cnc(N)c1Br
1584659          COC(=O)c1cc(CNC(=O)OC(C)(C)C)ccc1C
1584660                      NC(=O)c1ccc2ccccc2c1Br
1584661    CC(=O)Nc1cccc(-c2nc3cc(C)ccc3[nH]c2=O)c1
1584662     CC(NC(=O)OC(C)(C)C)c1nc(CO)nn1Cc1ccccc1
Length: 1584663, dtype: object

In [13]:
for i, (with_bos, with_eos, lengths) in enumerate(train_dataloader):
    print(with_bos.shape, with_eos.shape, len(lengths))
    if i == 2:
        break

torch.Size([50, 512]) torch.Size([50, 512]) 512
torch.Size([49, 512]) torch.Size([49, 512]) 512
torch.Size([52, 512]) torch.Size([52, 512]) 512


In [5]:
type(train), type(vocab), type(train_dataset), type(train_dataloader)

(numpy.ndarray,
 moses.utils.CharVocab,
 moses.utils.StringDataset,
 torch.utils.data.dataloader.DataLoader)

In [8]:
train.shape

(1584663,)

In [ ]:
from torch.utils.data import DataLoader
from moses import CharVocab, StringDataset

train = moses.get_dataset('train')
vocab = CharVocab.from_data(train)
train_dataset = StringDataset(vocab, train)
train_dataloader = DataLoader(
    train_dataset, batch_size=512,
    shuffle=True, collate_fn=train_dataset.default_collate
)

for with_bos, with_eos, lengths in train_dataloader:
    ...

In [2]:
from envs import molecular_metrics as mm

In [37]:
mm.MolecularMetrics._compute_SAS(mol)

1.5149307387756714

In [6]:
smiles_list[18]

'Cc1c(NC(=O)CSc2nc3sc4c(c3c(=O)[nH]2)CCCC4)c(=O)n(-c2ccccc2)n1C'

In [10]:
from rdkit import Chem
from envs.sascorer import calculateScore

smiles1 = "CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O"
mol1 = Chem.MolFromSmiles(smiles1)
sa_score1 = calculateScore(mol1)
print(f"SA Score для {smiles1}: {sa_score1}")

smiles2 = "Cc1c(NC(=O)CSc2nc3sc4c(c3c(=O)[nH]2)CCCC4)c(=O)n(-c2ccccc2)n1C"
mol2 = Chem.MolFromSmiles(smiles2)
sa_score2 = calculateScore(mol2)
print(f"SA Score для {smiles2}: {sa_score2}")

SA Score для CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O: 3.3559737244164882
SA Score для Cc1c(NC(=O)CSc2nc3sc4c(c3c(=O)[nH]2)CCCC4)c(=O)n(-c2ccccc2)n1C: 2.62217770660925


In [15]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import rdkit
import rdkit.Chem

fp1 = AllChem.GetMorganFingerprint(mol1, 2)
fp2 = AllChem.GetMorganFingerprint(mol2, 2)

sim = DataStructs.TanimotoSimilarity(fp1, fp2)
print('Танимото схожесть: {}'.format(sim))

Танимото схожесть: 0.13178294573643412


In [5]:
fp = open('./dataset/zinc250k/zinc250k.smi', 'r')
smiles_list = [smiles.strip() for smiles in fp]

In [9]:
valid_mol(mol)

In [7]:
mol = Chem.MolFromSmiles('C1=CC=CC=C1')
sml = Chem.MolToSmiles(mol)
print(sml)

c1ccccc1


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

# Пример SMILES
smiles = 'CC(=O)O'  # бензол

# Создание молекулы из SMILES
mol = Chem.MolFromSmiles(smiles)

# Рисование молекулы и сохранение изображения
img = Draw.MolToFile(mol, 'molecule.png')
